# Run the model on all test videos

In [1]:
# Setup the variables

test_folder = "data/SoccerNet/SN-GSR-2025/test"
model = "dfine"  # "yolo" or "dfine"

In [2]:
# Test if service is running
import requests

response = requests.get("http://localhost:4321/")
print(response.json())

{'message': 'Football Referee Evaluation API is running'}


In [3]:
def make_post_request(endpoint: str, body: dict) -> dict:
    response = requests.post(f"http://localhost:4321/{endpoint}", json=body)
    if response.status_code != 200:
        raise Exception(f"Request failed with status code {response.status_code}")
    return response.json()


### 1) Create a video from the frames

In [4]:
from src.utils.frames_to_mp4 import create_video_from_frames
from tqdm import tqdm
import os

for video_id in tqdm(os.listdir(test_folder), desc="Creating videos"):
    if video_id.startswith("SNGS"):
        frames_folder = os.path.join(test_folder, video_id, "img1")
        video_path = os.path.join(test_folder, video_id, f"{video_id}.mp4")
        if not os.path.exists(video_path):
            create_video_from_frames(frames_folder, video_path, fps=25)

Creating videos: 100%|██████████| 51/51 [00:00<00:00, 89464.45it/s]


### 2) Track the players

In [5]:
from typing import Literal

def track_players(video_path: str, model: Literal["yolo", "d-fine"], results_path: str | None = None):
    body = {
        "video_path": video_path,
        "model": model,
        "results_path": results_path
    }
    make_post_request("track", body)

In [6]:
from tqdm import tqdm
import os

for video_id in tqdm(os.listdir(test_folder), desc="Tracking players"):
    if video_id.startswith("SNGS"):

        video_path = os.path.join(test_folder, video_id, f"{video_id}.mp4")
        results_path = os.path.join(test_folder.replace("data", "data/predictions"), video_id, f"player_tracker_{model}")

        if not os.path.exists(video_path):
            print(f"Video file not found for video {video_id}")
            continue

        if os.path.exists(results_path):
            # already processed
            continue

        track_players(video_path, model=model, results_path=results_path)

Tracking players:  41%|████      | 21/51 [2:44:23<3:54:50, 469.69s/it]


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

### 3) Assign colors to each tracked player

In [48]:
import requests
from typing import Any

def assign_colors(video_path: str, detections: list[dict[str, Any]], results_path: str | None = None):
    body = {
        "video_path": video_path,
        "detections": detections,
        "results_path": results_path
    }
    make_post_request("assign-colors", body)

In [50]:
from tqdm import tqdm
import os
import json


for video_id in tqdm(os.listdir(test_folder), desc="Creating videos"):
    if video_id.startswith("SNGS"):
        video_path = os.path.join(test_folder, video_id, f"{video_id}.mp4")
        detections_path = os.path.join(test_folder.replace("data", "data/predictions"), video_id, f"player_tracker_{model}", "detections.json")
        results_path = os.path.join(test_folder.replace("data", "data/predictions"), video_id, f"color_assigner_{model}")

        if not os.path.exists(video_path):
            print(f"Video file not found for video {video_id}")
            continue

        if not os.path.exists(detections_path):
            print(f"Detections file not found for video {video_id}")
            continue

        with open(detections_path, "r") as f:
            detections = json.load(f)

        if os.path.exists(results_path):
            # already processed
            continue

        assign_colors(video_path, detections, results_path)

Creating videos: 100%|██████████| 51/51 [32:38<00:00, 38.40s/it]
